In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [9]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect

In [2]:
eng = create_engine("sqlite:///Resources/hawaii.sqlite")
s = Session(bind=eng)

In [3]:
# reflect an existing database into a new model
B = automap_base()
B.prepare(eng, reflect=True)
# reflect the tables

In [10]:
# We can view all of the classes that automap found
insp = inspect(eng) 
print(insp.get_table_names())

['measurement', 'station']


In [15]:
print("measurement---")
cols = insp.get_columns('measurement')
for c in cols:
    print(c["name"], c["type"])

print("\nstation---")
cols = insp.get_columns('station')
for c in cols:
    print(c["name"], c["type"])

measurement---
id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT

station---
id INTEGER
station TEXT
name TEXT
latitude FLOAT
longitude FLOAT
elevation FLOAT


In [45]:
# Save references to each table
meas = B.classes.measurement
stat = B.classes.station

d = dict()
sel = [stat.name, meas.date, meas.prcp]
for x in s.query(*sel).filter(meas.station == stat.station).order_by(meas.date.desc()).all():
    if x[0].split(',')[0] in d:
        d[x[0].split(',')[0]][x[1]] = x[2]
    else:
        d[x[0].split(',')[0]] = dict()
        d[x[0].split(',')[0]][x[1]] = x[2]
d

{'WAIKIKI 717.2': {'2017-08-23': 0.0,
  '2017-08-22': 0.0,
  '2017-08-21': 0.0,
  '2017-08-20': 0.0,
  '2017-08-19': 0.0,
  '2017-08-18': 0.0,
  '2017-08-15': 0.02,
  '2017-08-14': 0.0,
  '2017-08-13': 0.0,
  '2017-08-12': 0.0,
  '2017-08-11': 0.0,
  '2017-08-10': 0.0,
  '2017-08-09': 0.0,
  '2017-08-08': 0.0,
  '2017-08-07': 0.0,
  '2017-08-06': 0.0,
  '2017-08-05': 0.0,
  '2017-08-04': 0.02,
  '2017-08-03': 0.0,
  '2017-08-02': 0.0,
  '2017-08-01': 0.02,
  '2017-07-31': 0.0,
  '2017-07-30': 0.0,
  '2017-07-29': 0.0,
  '2017-07-28': 0.0,
  '2017-07-27': 0.0,
  '2017-07-26': 0.0,
  '2017-07-25': 0.0,
  '2017-07-24': 0.05,
  '2017-07-23': 0.0,
  '2017-07-22': 0.03,
  '2017-07-21': 0.0,
  '2017-07-20': 0.03,
  '2017-07-18': 0.05,
  '2017-07-17': 0.03,
  '2017-07-16': 0.02,
  '2017-07-15': 0.0,
  '2017-07-14': 0.02,
  '2017-07-13': 0.07,
  '2017-07-12': 0.0,
  '2017-07-11': 0.0,
  '2017-07-10': 0.0,
  '2017-07-09': 0.0,
  '2017-07-08': 0.0,
  '2017-07-07': 0.0,
  '2017-07-06': 0.0,
  '201

In [49]:
res = dict()
recent = date(1000,1,1)

sel = [meas.date, func.sum(meas.prcp)]
for x in s.query(*sel).group_by(meas.date).order_by(meas.date.desc()).all():
    cur = date(*convert_date(x[0]))
    if  cur > recent:
        recent = cur
    elif recent - timedelta(days=365) >= cur:
        break

    res[x[0]] = x[1]
res

{'2017-08-23': 0.53,
 '2017-08-22': 0.5,
 '2017-08-21': 0.5800000000000001,
 '2017-08-20': 0.01,
 '2017-08-19': 0.09,
 '2017-08-18': 0.06,
 '2017-08-17': 0.19,
 '2017-08-16': 0.61,
 '2017-08-15': 0.8200000000000001,
 '2017-08-14': 0.31,
 '2017-08-13': 0.0,
 '2017-08-12': 0.14,
 '2017-08-11': 0.0,
 '2017-08-10': 0.07,
 '2017-08-09': 0.15,
 '2017-08-08': 0.44000000000000006,
 '2017-08-07': 0.05,
 '2017-08-06': 0.0,
 '2017-08-05': 0.06,
 '2017-08-04': 0.06,
 '2017-08-03': 0.06999999999999999,
 '2017-08-02': 0.3,
 '2017-08-01': 0.13999999999999999,
 '2017-07-31': 0.0,
 '2017-07-30': 0.3,
 '2017-07-29': 0.61,
 '2017-07-28': 0.77,
 '2017-07-27': 0.01,
 '2017-07-26': 0.5,
 '2017-07-25': 0.61,
 '2017-07-24': 3.2699999999999996,
 '2017-07-23': 1.1300000000000001,
 '2017-07-22': 4.42,
 '2017-07-21': 0.13,
 '2017-07-20': 1.24,
 '2017-07-19': 0.41000000000000003,
 '2017-07-18': 2.53,
 '2017-07-17': 0.91,
 '2017-07-16': 0.81,
 '2017-07-15': 0.19,
 '2017-07-14': 0.9500000000000001,
 '2017-07-13': 1.

In [56]:
s.query(stat.name,stat.station).all()

[('WAIKIKI 717.2, HI US', 'USC00519397'),
 ('KANEOHE 838.1, HI US', 'USC00513117'),
 ('KUALOA RANCH HEADQUARTERS 886.9, HI US', 'USC00514830'),
 ('PEARL CITY, HI US', 'USC00517948'),
 ('UPPER WAHIAWA 874.3, HI US', 'USC00518838'),
 ('WAIMANALO EXPERIMENTAL FARM, HI US', 'USC00519523'),
 ('WAIHEE 837.5, HI US', 'USC00519281'),
 ('HONOLULU OBSERVATORY 702.2, HI US', 'USC00511918'),
 ('MANOA LYON ARBO 785.2, HI US', 'USC00516128')]

In [54]:
for e in d:
    print(e, len(d[e]))

WAIKIKI 717.2 2724
KUALOA RANCH HEADQUARTERS 886.9 2202
WAIMANALO EXPERIMENTAL FARM 2669
MANOA LYON ARBO 785.2 2612
WAIHEE 837.5 2772
KANEOHE 838.1 2709
PEARL CITY 1372
UPPER WAHIAWA 874.3 511
HONOLULU OBSERVATORY 702.2 1979


In [58]:
sel = [meas.date, meas.tobs]
for x in s.query(*sel).filter(meas.station == 'USC00519281').group_by(meas.date).order_by(meas.date.desc()).all():
    print(x)

('2017-08-18', 79.0)
('2017-08-17', 76.0)
('2017-08-16', 76.0)
('2017-08-15', 77.0)
('2017-08-14', 77.0)
('2017-08-13', 77.0)
('2017-08-06', 83.0)
('2017-08-05', 82.0)
('2017-08-04', 77.0)
('2017-07-31', 76.0)
('2017-07-30', 81.0)
('2017-07-29', 82.0)
('2017-07-28', 81.0)
('2017-07-27', 76.0)
('2017-07-26', 75.0)
('2017-07-25', 77.0)
('2017-07-24', 75.0)
('2017-07-23', 82.0)
('2017-07-22', 77.0)
('2017-07-21', 77.0)
('2017-07-20', 77.0)
('2017-07-19', 76.0)
('2017-07-18', 76.0)
('2017-07-17', 76.0)
('2017-07-16', 80.0)
('2017-07-15', 80.0)
('2017-07-14', 76.0)
('2017-07-13', 74.0)
('2017-07-12', 74.0)
('2017-07-11', 72.0)
('2017-07-10', 69.0)
('2017-07-09', 78.0)
('2017-07-08', 78.0)
('2017-07-07', 75.0)
('2017-07-06', 74.0)
('2017-07-05', 77.0)
('2017-07-04', 78.0)
('2017-07-03', 76.0)
('2017-07-02', 81.0)
('2017-07-01', 79.0)
('2017-06-30', 76.0)
('2017-06-29', 76.0)
('2017-06-28', 77.0)
('2017-06-27', 76.0)
('2017-06-26', 73.0)
('2017-06-25', 71.0)
('2017-06-24', 77.0)
('2017-06-23'

('2013-05-22', 75.0)
('2013-05-21', 69.0)
('2013-05-20', 69.0)
('2013-05-19', 78.0)
('2013-05-18', 81.0)
('2013-05-17', 73.0)
('2013-05-16', 73.0)
('2013-05-15', 74.0)
('2013-05-14', 73.0)
('2013-05-13', 73.0)
('2013-05-12', 71.0)
('2013-05-11', 74.0)
('2013-05-10', 71.0)
('2013-05-09', 67.0)
('2013-05-08', 65.0)
('2013-05-07', 64.0)
('2013-05-06', 69.0)
('2013-05-05', 75.0)
('2013-05-04', 76.0)
('2013-05-03', 70.0)
('2013-05-02', 73.0)
('2013-05-01', 70.0)
('2013-04-30', 67.0)
('2013-04-29', 74.0)
('2013-04-28', 79.0)
('2013-04-27', 75.0)
('2013-04-26', 71.0)
('2013-04-25', 71.0)
('2013-04-24', 68.0)
('2013-04-23', 74.0)
('2013-04-22', 70.0)
('2013-04-21', 76.0)
('2013-04-20', 68.0)
('2013-04-19', 72.0)
('2013-04-18', 71.0)
('2013-04-17', 70.0)
('2013-04-16', 63.0)
('2013-04-15', 70.0)
('2013-04-14', 70.0)
('2013-04-13', 74.0)
('2013-04-12', 73.0)
('2013-04-11', 74.0)
('2013-04-10', 75.0)
('2013-04-09', 71.0)
('2013-04-08', 71.0)
('2013-04-07', 75.0)
('2013-04-06', 73.0)
('2013-04-05'

('2010-07-16', 76.0)
('2010-07-15', 70.0)
('2010-07-14', 70.0)
('2010-07-13', 69.0)
('2010-07-12', 70.0)
('2010-07-11', 77.0)
('2010-07-10', 74.0)
('2010-07-09', 74.0)
('2010-07-08', 73.0)
('2010-07-07', 70.0)
('2010-07-06', 70.0)
('2010-07-05', 74.0)
('2010-07-04', 73.0)
('2010-07-03', 72.0)
('2010-07-02', 71.0)
('2010-07-01', 73.0)
('2010-06-30', 73.0)
('2010-06-29', 72.0)
('2010-06-28', 72.0)
('2010-06-27', 77.0)
('2010-06-26', 80.0)
('2010-06-25', 71.0)
('2010-06-24', 74.0)
('2010-06-23', 74.0)
('2010-06-22', 74.0)
('2010-06-21', 74.0)
('2010-06-20', 76.0)
('2010-06-19', 77.0)
('2010-06-18', 71.0)
('2010-06-17', 72.0)
('2010-06-16', 70.0)
('2010-06-15', 69.0)
('2010-06-14', 70.0)
('2010-06-13', 77.0)
('2010-06-12', 74.0)
('2010-06-11', 81.0)
('2010-06-10', 75.0)
('2010-06-09', 73.0)
('2010-06-08', 73.0)
('2010-06-07', 71.0)
('2010-06-06', 77.0)
('2010-06-05', 76.0)
('2010-06-04', 72.0)
('2010-06-03', 72.0)
('2010-06-02', 71.0)
('2010-06-01', 71.0)
('2010-05-31', 75.0)
('2010-05-30'

In [60]:
sel = [func.min(meas.tobs),func.avg(meas.tobs), func.max(meas.tobs)]
s.query(*sel).filter(meas.date > date(2016,8,23)).all()

[(58.0, 74.57894736842105, 87.0)]

In [ ]:
# Create our session (link) from Python to the DB


In [44]:
from datetime import date, timedelta
def convert_date(d):
    return [int(d.split("-")[0]),int(d.split("-")[1]),int(d.split("-")[2])]
date(*convert_date("2010-01-17")) - timedelta(days=365)

datetime.date(2009, 1, 17)

# Exploratory Climate Analysis

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results

# Calculate the date 1 year ago from the last data point in the database

# Perform a query to retrieve the data and precipitation scores

# Save the query results as a Pandas DataFrame and set the index to the date column

# Sort the dataframe by date

# Use Pandas Plotting with Matplotlib to plot the data


In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data

In [ ]:
# Design a query to show how many stations are available in this dataset?


In [ ]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.


In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature of the most active station?


In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


## Bonus Challenge Assignment

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.


In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)


In [ ]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation



In [ ]:
# Create a query that will calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()
    
daily_normals("01-01")

In [ ]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

# Set the start and end date of the trip

# Use the start and end date to create a range of dates

# Stip off the year and save a list of %m-%d strings

# Loop through the list of %m-%d strings and calculate the normals for each date


In [ ]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index


In [ ]:
# Plot the daily normals as an area plot with `stacked=False`
